In [62]:
import requests
import pandas as pd
from datetime import datetime, timedelta
from IPython.display import display

# Step 1: Retrieve exchange rate data from Frankfurter API for February 12-13, 2024
def get_exchange_rates(start_date, end_date):
    url = f"https://api.frankfurter.app/{start_date}..{end_date}?from=USD&to=EUR"
    response = requests.get(url)
    data = response.json()
    return data['rates']

# Dates for the exchange rates
start_date = '2024-02-12'
end_date = '2024-02-13'

exchange_rates = get_exchange_rates(start_date, end_date)

#output
print(exchange_rates)

{'2024-02-12': {'EUR': 0.92825}, '2024-02-13': {'EUR': 0.92653}}


In [63]:
# Step 2: Retrieve e-commerce data for 'Women’s Clothing' category
def get_products():
    ecommerce_api_url = 'https://fakestoreapi.com/products/category/women\'s clothing'
    response = requests.get(ecommerce_api_url)

    if response.status_code == 200:
        return response.json()
    else:
        return []

#response output
products = get_products()

df = pd.DataFrame(products)
df.rename(columns={'price': 'Price USD', 'title':'Title', 'id':'Id'}, inplace=True)
df['Exchange Date'] = end_date
df['Exchange Rate'] = df['Exchange Date'].apply(lambda x: exchange_rates[x]['EUR'])
df['Price EUR'] = df['Price USD'] * df['Exchange Rate']
df['Price EUR'] = df['Price EUR'].round(2)
df['Rating'] = df['rating'].apply(lambda x: x['rate'] if pd.notnull(x) else None)

df = df[['Id', 'Title', 'Rating', 'Price USD', 'Price EUR', 'Exchange Date']]

display(df)


,Id,Title,Rating,Price USD,Price EUR,Exchange Date
0,15,BIYLACLESEN Women's 3-in-1 Snowboard Jacket Wi...,2.6,56.99,52.80,2024-02-13
1,16,Lock and Love Women's Removable Hooded Faux Le...,2.9,29.95,27.75,2024-02-13
2,17,Rain Jacket Women Windbreaker Striped Climbing...,3.8,39.99,37.05,2024-02-13
3,18,MBJ Women's Solid Short Sleeve Boat Neck V,4.7,9.85,9.13,2024-02-13
4,19,Opna Women's Short Sleeve Moisture,4.5,7.95,7.37,2024-02-13
5,20,DANVOUY Womens T Shirt Casual Cotton Short,3.6,12.99,12.04,2024-02-13


In [64]:
""" create statement

CREATE TABLE IF NOT EXISTS my_dataset.top_5_products_each_day (
  Id INT64,
  Title STRING,
  Price_USD FLOAT64,
  Price_EUR FLOAT64,
  Exchange_Date DATE,
  Rating FLOAT64
); """


top_5_df = df.sort_values(['Exchange Date', 'Rating'], ascending=[True, False]).groupby('Exchange Date').head(5)
display(top_5_df)

"""
#insert_method, if google bigquery should be something like:
from google.cloud import bigquery
from google.oauth2 import service_account

# Create a BigQuery client
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

# Define the table where you want to insert data
table_id = "my_project.my_dataset.top_5_products_each_day"

# Load the DataFrame into BigQuery
job = client.load_table_from_dataframe(top_5_df, table_id) """

,Id,Title,Rating,Price USD,Price EUR,Exchange Date
3,18,MBJ Women's Solid Short Sleeve Boat Neck V,4.7,9.85,9.13,2024-02-13
4,19,Opna Women's Short Sleeve Moisture,4.5,7.95,7.37,2024-02-13
2,17,Rain Jacket Women Windbreaker Striped Climbing...,3.8,39.99,37.05,2024-02-13
5,20,DANVOUY Womens T Shirt Casual Cotton Short,3.6,12.99,12.04,2024-02-13
1,16,Lock and Love Women's Removable Hooded Faux Le...,2.9,29.95,27.75,2024-02-13


' from google.cloud import bigquery\nfrom google.oauth2 import service_account\n\n# Create a BigQuery client\nclient = bigquery.Client(credentials=credentials, project=credentials.project_id)\n\n# Define the table where you want to insert data\ntable_id = "my_project.my_dataset.top_5_products_each_day"\n\n# Load the DataFrame into BigQuery\njob = client.load_table_from_dataframe(top_5_df, table_id) '